In [1]:
import os
import requests

Downloading The Document (PDF File)

In [4]:
# get document name
pdf_name = "human-nutrition-text.pdf"

if not os.path.exists(pdf_name):
    print(f"Downloading {pdf_name}")

    # url of document to download

    pdf_url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    resp = requests.get(pdf_url)

    if resp.status_code == 200:
        with open(pdf_name, "wb") as f:
            f.write(resp.content)
        print(f"Downloaded {pdf_name} and Saved")
    else:
        print(f"Failed to download {pdf_name}")
else:
    print(f"{pdf_name} already exists")


Downloaded human-nutrition-text.pdf and Saved


Open the PDF

In [6]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.6 MB/s eta 0:00:00


In [7]:
import fitz  # PyMuPDF !pip install pymupdf
from tqdm.auto import tqdm

def text_formatter(text:str)->str:
    """perform basic text preprocessing. """

    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text


def open_and_read_pdf(pdf_name:str)->list[dict]:
    doc = fitz.open(pdf_name)
    pages_and_textx = []

    for page_no,page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_textx.append({
            "page_number" : page_no+1,
            "page_char_count" : len(text),
            "page_word_count" : len(text.split()),
            "page_sentence_count_raw" : len(text.split(". ")),
            "page_token_count" : len(text)/4,
            "text" : text
            })
    return pages_and_textx

pages_and_texts = open_and_read_pdf(pdf_name)

0it [00:00, ?it/s]

In [9]:
import random

random.sample(pages_and_texts,k=3)

[{'page_number': 480,
  'page_char_count': 1749,
  'page_word_count': 262,
  'page_sentence_count_raw': 23,
  'page_token_count': 437.25,
  'text': 'your body, including your brain cells. This is the cause of the so- called “morning hangover”. Do not take Tylenol (acetaminophen).  Alcohol  metabolism  activates  an  enzyme  that  transforms  acetaminophen  into  a  toxic  metabolite  that  causes  liver  inflammation and damage. Liver damage may not be irreversible.  Instead, drink water with electrolytes or sport drinks to rehydrate  the body’s cells.  Alcohol is a volatile (flammable) organic substance and can be  converted to a gas. The lungs exhale alcohol as a gas. The more  alcohol consumed, the stronger the smell of alcohol in a person’s  breathe. Breathalyzer tests measure the exhaled alcohol levels in the  lungs to determine the state of inebriation.  The liver metabolizes up to 85% – 98% of the circulatory ethanol.  The liver uses two metabolic processes to get rid of this ci

In [10]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,29,4,1,7.25,Human Nutrition: 2020 Edition
1,2,0,0,1,0.00,
2,3,320,42,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,4,212,30,1,53.00,Human Nutrition: 2020 Edition by University of...
4,5,797,116,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [11]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035
std,348.86387,560.382275,86.491465,6.187226,140.095569
min,1.00000,0.000000,0.000000,1.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000


Why we need token Count?

1. Embeding Model don't deal with infinite tokens.
2. LLMs don't deal with lnfinite tokens.

# Splitting Pages into sentences